In [4]:
from LandBase import *

# 주피터노트북 매직커맨드 혹은 매직키워드
# 구글에서 찾을 때는 "jupyter notebook Built-in magic commands"라고 찾으면 수월함
# %lsmagic : 사용 가능한 매직커맨드 목록
# %quickref : 사용 가능한 매직커맨드 목록과 설명 (quick reference)
# %prun : 주피터노트북에서 실행된 내용의 퍼포먼스 측정
# %time : 주피터노트북에서 실행된 내용의 시간 측정
# %pdb : 디버깅 목적으로 오류가 발생하면 그 위치에서 멈추게 함
# %load_ext : 외부 플러그인 가져오기

'''분양목록
http://www.drapt.com/e_sale/index.htm?page_name=cal&menu_key=0&item=3&view_type=month&today_date=2015-10
얻을 수 있는 정보: 단지이름, 단지번호

단지기본 정보
http://www.drapt.com/sise/html_view5/total.php?no=53685
얻을 수 있는 정보: 주소, 단지규모, 입주시기, 난방방식, 시공사, 특징

단지면적 정보
http://www.drapt.com/sise/html_view5/size_info.php?no=53685
얻을 수 있는 정보: 공급면적, 전용면적, 매매가, 전세가, 방/욕실, 가구수(임대)
'''
def apt_sell_info(p_ym='201510') :
    ####
    # 분양정보목록
    # 출처: http://www.drapt.com/e_sale/index.htm?page_name=cal&menu_key=0&item=3&view_type=month&today_date=년-월
    ####
    
    r_list = []
    url = 'http://www.drapt.com/e_sale/index.htm?page_name=cal&menu_key=0&item=3&view_type=month&today_date=' + p_ym[:4] + '-' + p_ym[4:]
    print(url)
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml') # bs4.BeautifulSoup 타입
    # bs4.BeautifulSoup 타입에서 find 혹은 find_all만 한 결과는 bs4.element.ResultSet 타입이다.
    # soup.find_all('table', 'esale_cal_table3') = soup.find_all('table', class_='esale_cal_table3')
    # soup.find_all('table', class_='esale_cal_table3') : table 태그 중에서 class가 esale_cal_table3인 경우를 모두 찾음, bs4.element.ResultSet 타입
    # soup.find('table', 'esale_cal_table3').find_all('tr') : 위에서 찾은 결과에서 tr 태그를 가진 값 모두 찾음, bs4.element.ResultSet 타입
    for tr in soup.find('table', class_='esale_cal_table3').find_all('tr') :
        # tr.find('th').find('span', 'td_date').find('strong') : 위에 찾은 결과에서 th 태그 -> span태그 & class가 td_date이고 -> strong 태그
        # tr.find('th').find('span', 'td_date').find('strong').string : 위 결과에서 문자열로 된 값만 추출, bs4.element.NavigableString 타입, 여기서는 필요없으므로 안 찾음
        for li in tr.find_all('li', class_=False) :
            분양유형 = '민간분양'
            형태 = li.find('span').string.strip('[]')
            if 형태 == '오피스텔' : 
                continue # 오피스텔은 건너뛰기
            apt_nm = li.find('a')['title'].strip()
            if not apt_nm.startswith('서울') and not apt_nm.startswith('인천') and not apt_nm.startswith('경기') :
                continue # 서울, 인천, 경기 아니면 건너뛰기
            apt_no = re.findall('\d+', li.find('a')['href'])[0]
            if apt_nm.count('뉴스테이') :
                분양유형 = '뉴스테이'
                apt_nm = apt_nm.replace('뉴스테이', '')
            elif apt_nm.count('공공임대') :
                분양유형 = '공공임대'
                apt_nm = apt_nm.replace('공공임대', '')
            elif apt_nm.count('영구임대') :
                분양유형 = '영구임대'
                apt_nm = apt_nm.replace('영구임대', '')
            elif apt_nm.count('국민임대') :
                분양유형 = '국민임대'
                apt_nm = apt_nm.replace('국민임대', '')
            elif apt_nm.count('공공분양') :
                분양유형 = '공공분양'
                apt_nm = apt_nm.replace('공공분양', '')
            if apt_nm.endswith('순위') :
                apt_nm = apt_nm.rsplit(' ', 1)[0] # 오른쪽부터 1번만 분리
            apt_nm = apt_nm.replace('()', '').replace(',)', ')').replace(', )', ')')
            
            r_dict = apt_basic_info(apt_no) # 단지 기본정보 함수 호출
            if '입주시기' not in r_dict.keys() :
                r_dict['입주시기'] = str(int(p_ym[:4])+3) + p_ym[4:]
            if '특징' not in r_dict.keys() :
                r_dict['특징'] = ''
            
            r_list.append([p_ym, r_dict['입주시기'], 분양유형, 형태, apt_no, apt_nm, r_dict['주소'], r_dict['동수'], r_dict['가구수'], r_dict['최고층'], r_dict['특징']])
    
    df = pd.DataFrame(r_list, columns=['sell_ym', 'move_ym', 'sell_type', 'apt_type', 'apt_no', 'apt_nm', 'address', 'building_cnt', 'house_cnt', 'top_floor', 'characteristics'])
    df.drop_duplicates(inplace=True)
    return df
    
def apt_basic_info(p_no='39801') :
    ####
    # 단지기본 정보
    # 출처: http://www.drapt.com/sise/html_view5/total.php?no=단지번호
    # 정보: 최고층, 동수, 가구수, 입주시기, 주소, 특징
    ####
    
    r_dict = {'no':p_no}
    url = 'http://www.drapt.com/sise/html_view5/total.php?no=' + p_no

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml') # bs4.BeautifulSoup 타입
    
    for tr in soup.find('table').find_all('tr') :
        th = tr.find_all('th')
        td = tr.find_all('td')
        for i in range(len(th)) :
            th_str = th[i].string.replace('\n', '').strip()
            td_str = td[i].contents[0].string.replace('\n', '').strip()
            if th_str == '단지규모' :
                r_dict['최고층'] = td_str.split()[0].split('~')[-1].replace('층', '')
                r_dict['동수'] = td_str.split()[1].replace('개동', '')
                r_dict['가구수'] = td_str.split()[-1].replace('총', '').replace('가구', '')
            elif th_str == '입주시기' :
                r_dict[th_str] = td_str.replace('년', '').replace('월', '').replace(' ', '')[:6]
            elif th_str in ['주소', '특징'] :
                r_dict[th_str] = td_str
    
    #display(r_dict)
    return r_dict

# 아파트 단지 코드/이름
def get_naver_complex(area_code):
    
    url = 'http://land.naver.com/article/articleList.nhn?' \
        + 'rletTypeCd=A01&tradeTypeCd=A1&hscpTypeCd=A01%3AA03%3AA04' \
        + '&cortarNo=' + area_code

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    
    value_list = []

    apt_complex = soup.find('select', {'title':'단지'}).find_all('option')
    for apt in apt_complex:
        if apt.text != '단지' and apt['value'] != 'none' :
            value_list.append([apt['value'], apt.text])
                  
    cols = ['단지번호', '단지명']
    df = pd.DataFrame(value_list, columns=cols)
    return df

# 아파트 단지 정보
def get_naver_aptinfo(area_code, rletNo, apt_name):

    trs = table.tbody.find_all('tr')
    
    aptinfo = {}
    for tr in trs :
        th = tr.find_all('th')
        td = tr.find_all('td')
        try:
            aptinfo[th[0].text] = td[0].text
            aptinfo[th[1].text] = td[1].text
        except:
            pass

    # 100 세대 미만은 무조건 버림
    if int(re.findall('[\d]+', aptinfo['총세대수'])[0]) < 100 :
        return pd.DataFrame()

    value_list = []
    value_list.append(apt_name)
    value_list.append(re.findall('[\d]+', aptinfo['총세대수'])[0])
    value_list.append(aptinfo['총동수'].replace('개동', '').strip())
    value_list = [value_list]

    cols = ['lawd_cd', 'rlet_no', 'apt_name', 'house_cnt', 'building_cnt', 'build_ym', 'company', 'park_total', 'park_house', 'heat_type', 'heat_fuel', 'floor_area_ratio', 'coverage_ratio', 'top_floor', 'low_floor']
    df = pd.DataFrame(value_list, columns=cols)
    return df

In [5]:
# DB Connection Open
#con = get_db_connection()

# 
df = apt_sell_info('201510')
display(df)

# DB Connection Close
#disconnect_db(con)

print('Program Finised')

http://www.drapt.com/e_sale/index.htm?page_name=cal&menu_key=0&item=3&view_type=month&today_date=2015-10


,sell_ym,move_ym,sell_type,apt_type,apt_no,apt_nm,address,building_cnt,house_cnt,top_floor,characteristics
0,201510,201810,뉴스테이,임대,39801,경기도 수원시 오목천동 수원권선 꿈에그린,경기도 수원시 오목천동 824-1,32,2400,20,상수천 조망권/
1,201510,201606,영구임대,임대,53386,경기도 시흥시 목감동 시흥목감 A1블록,경기도 시흥시 목감동 시흥목감지구 A1블록,2,402,19,시흥목감 공공주택지구/
2,201510,201701,민간분양,아파트,53685,서울특별시 강동구 고덕동 고덕 래미안 힐스테이트,서울특별시 강동구 고덕동 670,51,3658,35,2014년 분양한 고덕래미안힐스트이트(고덕시영 재건축) 중 조합원 미계약 세대. 전...
3,201510,201712,민간분양,아파트,53684,경기도 남양주시 진건읍 다산신도시 아이파크(C3블록),경기도 남양주시 진건읍 진건지구 C3블록,5,467,29,다산진건공공주택지구/다산신도시 진건보금자리지구. 분양가상한제 & 전매제한(1년) 적용./
4,201510,201712,공공임대,임대,53377,경기도 수원시 호매실동 수원호매실지구 B3블록,경기도 수원시 호매실동 호매실지구 B3블록,13,1300,25,호매실공공주택지구
8,201510,201606,국민임대,임대,53741,경기도 화성시 남양읍 화성 남양뉴타운 A3블록,경기도 화성시 남양읍 화성 남양뉴타운 도시개발구역 A3블록,6,876,25,화성 남양/
9,201510,201610,국민임대,임대,53363,경기도 부천시 옥길동 부천옥길지구 A1블록,경기도 부천시 옥길동 부천옥길보금자리지구 A1블록,8,916,28,옥길지구/
13,201510,201803,민간분양,아파트,53739,서울특별시 서초구 반포동 반포 센트럴 푸르지오 써밋,서울특별시 서초구 반포동 30-20 (삼호가든4차 재건축),8,751,35,전매제한(6개월)/
15,201510,201802,민간분양,아파트,50982,경기도 평택시 장당동 평택 장당 제일풍경채 3차,경기도 평택시 장당동 463,7,448,24,민간택지(분양가상한제 미적용). 전매제한 6개월/
16,201510,201805,민간분양,아파트,53542,경기도 화성시 남양읍 화성 남양뉴타운 동광뷰엘(B4블록),경기도 화성시 남양읍 남양뉴타운 B4블록,8,673,36,화성 남양/분양가상한제 적용. 전매제한(1년)/


Program Finised
